In [1]:
using SoftmaxClassifier
using Training
using AbstractTrees

In [2]:
distribution = Dict("A"=>0.4, "B" => 0.2, "C" => 0.2, "D"=> 0.1, "E"=>0.1)

Dict{String,Float64} with 5 entries:
  "B" => 0.2
  "A" => 0.4
  "C" => 0.2
  "D" => 0.1
  "E" => 0.1

In [3]:
classifier_tree,codebook = Training.initialize_network(distribution, 5, WordEmbeddings.huffman_tree)

(Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[0.63829 0.548069; -0.372887 -0.169173; 0.241831 -0.692348; -0.676522 0.988183; 0.598434 0.407975]),Dict("B"=>[2,1],"A"=>[1],"C"=>[2,2,1],"D"=>[2,2,2,1],"E"=>[2,2,2,2]))

In [4]:
print_tree(STDOUT, classifier_tree)

Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[0.63829 0.548069; -0.372887 -0.169173; 0.241831 -0.692348; -0.676522 0.988183; 0.598434 0.407975])
├─ Trees.BranchNode with 0 children. data = "A"
└─ Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[0.862341 -0.506161; 0.248016 0.163408; 0.522552 -0.618117; 0.0447176 0.81089; -0.903935 -0.330532])
   ├─ Trees.BranchNode with 0 children. data = "B"
   └─ Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[0.373455 -0.431678; 0.550001 0.548255; 0.701398 0.446863; -0.741222 -0.671272; 0.807268 -0.411665])
      ├─ Trees.BranchNode with 0 children. data = "C"
      └─ Trees.BranchNode with 2 children. data = SoftmaxClassifier.LinearClassifier{2}(5,Float32[0.770272 -0.965899; -0.680085 -0.13024; -0.252352 -0.26909; 0.862566 0.463776; 0.24174 -0.00312933])
         ├─ Trees.BranchNode with 0 children. data = "D"
         └─ Tree

In [5]:
f_A(x) = collect(predict(classifier_tree.data, x))[1]

f_A (generic function with 1 method)

In [6]:
@show predict(classifier_tree.data, x)
f_A([0.5f0, 0.5f0, 0.5f0, 0.5f0, 0.5f0])

LoadError: LoadError: UndefVarError: x not defined
while loading In[6], in expression starting on line 200

In [7]:
using ForwardDiff

In [8]:
x = [0.5f0, 0.5f0, 0.5f0, 0.5f0, 0.5f0]

@show y=f_A(x)


ForwardDiff.gradient(f_A, x)

y = f_A(x) = 0.4190243f0


5-element Array{Float32,1}:
  0.0219637
 -0.0495929
  0.227419 
 -0.405261 
  0.046366 

In [9]:
l_A(x) = log(collect(predict(classifier_tree.data, x))[1])
@time ForwardDiff.gradient(l_A, x; )

  0.048719 seconds (11.48 k allocations: 527.631 KB)


5-element Array{Float32,1}:
  0.0524163
 -0.118353 
  0.542735 
 -0.967153 
  0.110652 

In [10]:
l_B(x) = log(collect(predict(classifier_tree.data, x))[2])
ForwardDiff.gradient(l_B, x; )

5-element Array{Float32,1}:
 -0.0378048
  0.0853613
 -0.391444 
  0.697552 
 -0.0798071

In [11]:
function calculate_input_gradient!{F1<:Number, F2<:Number,K}(
                    c::LinearClassifier{K},
                    x::AbstractVector{F1},
                    y::Int64,
                    input_gradient::AbstractVector{F2})
    #input_gradient += -ForwardDiff.gradient(log(collect(predict(c, x))[y]), x)
    outputs = collect(predict(c, x))
    outputs[y] -= 1

    for i in 1:K
        m = outputs[i]
        for j in 1:c.n
            input_gradient[j] -= outputs[i] * c.weights[j, i]
        end
    end
    input_gradient
end


calculate_input_gradient! (generic function with 1 method)

In [12]:
using Optim

In [13]:
using WordEmbeddings
using Query
using JLD

In [14]:
ee = load("../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld","ee");
#ee = restore("../eval/models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model")
#dtree,labels = nn_using AbstractTreestree(ee)
#""

Word embedding(dimension = 50)of 88262 words


In [22]:
reload("Query")

In [72]:
function get_optimisable(context, embed)
    function loss{F}(x::Vector{F})
        total_lp = zero(F)
        outputs = Vector{F}(2)
        input_gradient = similar(x)
        for target_word in context
            !haskey(embed.codebook, target_word) && continue          
            word_lp  = zero(F)
            node = embed.classification_tree
            for y in embed.codebook[target_word]
                c = node.data
                outputs[1],outputs[2] = predict(c, x)
                lp = log(outputs[y])
                @inbounds node = node.children[y]               
                word_lp += lp
            end
            total_lp += word_lp
        end
        total_lp  
    end
    
    @assert(size(ee.classification_tree.data.weights,2) == 2) #Only dealing with binary trees
    function loss_and_grad!{F}(x::Vector{F}, total_input_gradient)
        #Wrong
        total_lp = zero(F)
        outputs = Vector{F}(2)
        input_gradient = similar(x)
        for target_word in context
            !haskey(embed.codebook, target_word) && continue
            
            word_lp  = zero(F)
            fill!(input_gradient,0.0)
            
            node = embed.classification_tree
            for y in embed.codebook[target_word]
                c = node.data
                outputs[1],outputs[2] = predict(c, x)
                lp = log(outputs[y])
                
                outputs[y] -= 1                
                for j in 1:c.n
                    input_gradient[1] -= outputs[1] * c.weights[j, 1]/exp(lp)
                    input_gradient[2] -= outputs[2] * c.weights[j, 2]/exp(lp)
                end
                @inbounds node = node.children[y]               
                word_lp += lp
            end
            total_lp += word_lp
            total_input_gradient += input_gradient./exp(word_lp)
        end
        total_lp  
    end
    
    DifferentiableFunction(loss)#, loss_and_grad!, loss_and_grad!)
end

get_optimisable (generic function with 1 method)

In [82]:
word = "ordinary"
context = split("The weighted arithmetic mean is similar to an arithmetic mean")
x_outer = ee.embedding[word]
diff_fun = get_optimisable(context, ee)

Optim.DifferentiableFunction(loss,Optim.g!,Optim.fg!)

In [83]:
diff_fun.f(x_outer)


-76.35538f0

In [84]:
grad = similar(x_outer)
diff_fun.g!(x_outer, grad)
[grad ForwardDiff.gradient(diff_fun.f, x_outer)]


50×2 Array{Float32,2}:
 -4.18088    -4.18122  
  0.820054    0.819721 
 -3.25231    -3.25221  
  2.87019     2.87096  
  0.94562     0.945873 
  1.31922     1.31999  
  0.397625    0.398699 
 -3.47011    -3.47057  
  3.76078     3.76119  
 -1.7587     -1.75965  
  0.994451    0.995194 
  1.5533      1.55326  
 -0.499938   -0.49923  
 -2.72602    -2.72729  
  2.10207     2.10129  
 -4.49479    -4.49323  
  0.0333292   0.0331022
 -1.73855    -1.73951  
 -5.91787    -5.91708  
 -2.52837    -2.52777  
 -3.35773    -3.35704  
 -1.768      -1.7687   
  4.88234     4.88233  
 -1.81605    -1.81622  
  2.63843     2.63862  
 -0.51544    -0.515799 
 -6.02561    -6.02651  
  2.30049     2.30092  
 -1.65871    -1.65848  
  3.57165     3.57141  
 -4.07934    -4.08149  
  0.31779     0.317361 
 -1.93077    -1.93033  
  1.0208      1.02157  
 -3.55693    -3.5564   
  0.132542    0.13192  
 -0.230204   -0.22957  
 -0.274385   -0.275123 
  1.1673      1.1683   
 -0.577448   -0.576653 
 -1.3944     -1.3

In [85]:
@time optimize(diff_fun, x_outer)


  0.824189 seconds (5.08 M allocations: 102.147 MB, 10.81% gc time)


Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [0.26443565,-0.5347172, ...]
 * Minimizer: [0.95223916,-0.66962564, ...]
 * Minimum: -1.839834e+02
 * Iterations: 2
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: true
   * |g(x)| < 1.0e-08: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 64
 * Gradient Calls: 64

In [88]:
using NBInclude